<a href="https://colab.research.google.com/github/gowripreetham/SJSU_AutoGluon_Assignment/blob/main/Copy_of_ieee_fraud_autogluon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install -U pip
!pip -q install -U autogluon kaggle
import pandas as pd, numpy as np, os
from autogluon.tabular import TabularPredictor


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 63.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.8.0+cu126 requires torch==2.8.0, but you have torch 2.7.1 which is incompatible.


In [ ]:
from google.colab import files
files.upload()  # Choose the kaggle.json you just downloaded


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"gowripreetham","key":"71c1cdc7910384e1f4d54630648fdfdd"}'}

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets list -s fraud


ref                                                         title                                                   size  lastUpdated                 downloadCount  voteCount  usabilityRating  
----------------------------------------------------------  ------------------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
mlg-ulb/creditcardfraud                                     Credit Card Fraud Detection                         69155672  2018-03-23 01:17:27.913000        1016926      12568  0.85294116       
whenamancodes/fraud-detection                               Fraud Detection                                     69155672  2022-09-12 11:54:40.550000          11420        123  1.0              
dhanushnarayananr/credit-card-fraud                         Credit Card Fraud                                   30281243  2022-05-07 15:09:29.617000          29121        221  0.9411765        
ealaxi/paysim1                

In [ ]:
!kaggle competitions download -c ieee-fraud-detection -p ./ieee_fraud
!unzip -q ./ieee_fraud/ieee-fraud-detection.zip -d ./ieee_fraud


  0% 0.00/118M [00:00<?, ?B/s]
100% 118M/118M [00:00<00:00, 1.62GB/s]


In [ ]:
!ls -lh ./ieee_fraud


total 1.4G
-rw-r--r-- 1 root root 119M Dec 11  2019 ieee-fraud-detection.zip
-rw-r--r-- 1 root root 5.8M Dec 11  2019 sample_submission.csv
-rw-r--r-- 1 root root  25M Dec 11  2019 test_identity.csv
-rw-r--r-- 1 root root 585M Dec 11  2019 test_transaction.csv
-rw-r--r-- 1 root root  26M Dec 11  2019 train_identity.csv
-rw-r--r-- 1 root root 652M Dec 11  2019 train_transaction.csv


In [ ]:
directory = './ieee_fraud/'
train_identity = pd.read_csv(directory + 'train_identity.csv')
train_transaction = pd.read_csv(directory + 'train_transaction.csv')

# Use only first 5000 rows to keep it light
train_transaction = train_transaction.head(5000)
train_data = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')
train_data.head()


,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M


In [ ]:
label = 'isFraud'
save_path = directory + 'AutoGluonModels/'
predictor = TabularPredictor(label=label, eval_metric='roc_auc', path=save_path)

predictor.fit(train_data, presets='medium_quality_faster_train', time_limit=120)


Preset alias specified: 'medium_quality_faster_train' maps to 'medium_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Oct  2 10:42:05 UTC 2025
CPU Count:          2
Memory Avail:       7.72 GB / 12.67 GB (60.9%)
Disk Space Avail:   65.14 GB / 112.64 GB (57.8%)
Presets specified: ['medium_quality_faster_train']
Using hyperparameters preset: hyperparameters='default'
Beginning AutoGluon training ... Time limit = 120s
AutoGluon will save models to "/content/ieee_fraud/AutoGluonModels"
Train Data Rows:    5000
Train Data Columns: 433
Label Column:       isFraud
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [np.int64(0), np.int64(1)]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter 

In [ ]:
# Align test columns with training columns
missing_cols = [col for col in predictor.feature_metadata.get_features() if col not in test_data.columns]
for col in missing_cols:
    test_data[col] = 0  # or np.nan, but 0 works fine for numeric placeholders

# Reorder columns to match training order
test_data = test_data[predictor.feature_metadata.get_features()]

# Now run predictions safely
y_predproba = predictor.predict_proba(test_data, as_multiclass=False)
y_predproba.head()


,isFraud
0,0.213135
1,0.216426
2,0.229741
3,0.214279
4,0.214320


In [ ]:
# Load test data
test_identity = pd.read_csv(directory + 'test_identity.csv')
test_transaction = pd.read_csv(directory + 'test_transaction.csv')

# Use a small subset for speed
test_transaction = test_transaction.head(5000)

# Merge test data same as training
test_data = pd.merge(test_transaction, test_identity, on='TransactionID', how='left')

# --- UNIVERSAL FIX: Align test columns with training columns ---
# Get the columns AutoGluon saw during training
train_features = predictor.feature_metadata.get_features()  # works across versions

# Identify missing columns
missing_cols = [col for col in train_features if col not in test_data.columns]

# Add missing columns with 0 (safe placeholder)
for col in missing_cols:
    test_data[col] = 0

# Add any extra test columns not seen in training (drop them)
extra_cols = [col for col in test_data.columns if col not in train_features]
if extra_cols:
    test_data = test_data.drop(columns=extra_cols)

# Ensure column order matches training
test_data = test_data[train_features]

print(f"✅ Added {len(missing_cols)} missing columns and dropped {len(extra_cols)} extras.")

# Now predict safely
y_predproba = predictor.predict_proba(test_data, as_multiclass=False)
y_predproba.head()


✅ Added 38 missing columns and dropped 56 extras.


,isFraud
0,0.213135
1,0.216426
2,0.229741
3,0.214279
4,0.214320


In [ ]:
submission = pd.read_csv(directory + 'sample_submission.csv')
submission = submission.head(len(y_predproba))
submission['isFraud'] = y_predproba.values
submission.to_csv(directory + 'my_submission.csv', index=False)

print("Saved:", directory + "my_submission.csv")
submission.head()


Saved: ./ieee_fraud/my_submission.csv


,TransactionID,isFraud
0,3663549,0.213135
1,3663550,0.216426
2,3663551,0.229741
3,3663552,0.214279
4,3663553,0.214320


### ✅ Summary
- Dataset: IEEE-CIS Fraud Detection (sample subset)
- Model: AutoGluon Tabular (`medium_quality_faster_train`)
- Metric: ROC-AUC
- Output: `my_submission.csv`
- All steps executed successfully in Colab with outputs saved.
